In [139]:
#import smodels modules

import numpy as np

from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor, TheoryPredictionsCombiner
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
import time

setLogLevel("info")

In [140]:
#import taco modules
import sys, os
sys.path.insert(0, os.path.expanduser("~/taco_code"))

import numpy as np
from codes.Full_SR_Ranking.pathfinder.path_finder import PathFinder

## Input array of True/False manually

In [141]:
listOfAna = ['ATLAS-SUSY-2018-05-ewk-eff', 'ATLAS-SUSY-2018-06-eff', 'ATLAS-SUSY-2018-32-eff',
             'ATLAS-SUSY-2018-41-eff','ATLAS-SUSY-2019-02-eff','ATLAS-SUSY-2019-08-eff', 'ATLAS-SUSY-2019-09-eff']
comb_matrix = np.array([[False,True,True,True,True,True,True],
                        [True,False,True,True,True,True,False],
                        [True,True,False,True,False,True,True], 
                        [True,True,True,False,True,True,True],
                        [True,True,False,True,False,True,True],
                        [True,True,True,True,True,False,True],
                        [True,False,True,True,True,True,False]])

np.array(~comb_matrix, dtype=int)

array([[1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1]])

## Finding Weights

In [142]:
slhafile = '/Users/sahananarasimha/smodels/ew_7q306a4m.slha'

In [143]:
def getWeightsFor(analyses, inputfile, expected=True):
    
    sigmacut = 0.005*fb
    mingap = 5.*GeV

    database = Database('/Users/sahananarasimha/smodels-database/13TeV/ATLAS/%s'%analyses)
    
    model = Model(BSMparticles = BSMList, SMparticles = SMList)
    model.updateParticles(inputFile = slhafile)
    
    toplist = decomposer.decompose(model, sigmacut, doCompress=True, doInvisible=True, minmassgap=mingap)
    print("\n ", analyses)
    exp = database.getExpResults()[0]
    
    predictions = theoryPredictionsFor(exp, toplist, combinedResults=True)
    
    if not predictions:
        print("\n weight =", predictions)
        return None  
    
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset

        if dataset.getType() == 'combined' or dataset.getType() == 'efficiencyMap':
            #get expected llhd
            if expected:
                #print('\n', datasetID, txnames, '\n')
                lbsm = theoryPrediction.likelihood(expected=True)
                lsm = theoryPrediction.lsm(expected=True)
                weight = -np.log(lbsm/lsm)  #returning nll ratio
                return weight
            
            #get observed llhd
            lbsm = theoryPrediction.likelihood()
            lsm = theoryPrediction.lsm()
            weight = -np.log(lbsm/lsm)  #returning nll ratio
            return weight
                

ana_weights = []
index = []

for ana in listOfAna:
    weight = getWeightsFor(analyses = ana, inputfile = slhafile)
    
    #Get index of analyses for which theoryPrediction is None
    if weight == None:
        index.append(listOfAna.index(ana))
        continue
        
    ana_weights.append(weight)

#Remove analysis from comb_matrix for which theoryPrediction is None
comb_matrix = np.delete(np.delete(comb_matrix, index,0), index,1)

INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2018-05-ewk-eff/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2018-05-ewk-eff/db31.pcl in 0.0 secs.
INFO in model.updateParticles() in 393: Loaded 62 BSM particles



  ATLAS-SUSY-2018-05-ewk-eff


INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2018-06-eff/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2018-06-eff/db31.pcl in 0.0 secs.
INFO in model.updateParticles() in 393: Loaded 62 BSM particles
INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2018-32-eff/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2018-32-eff/db31.pcl in 0.0 secs.
INFO in model.updateParticles() in 393: Loaded 62 BSM particles
INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2018-41-eff/db31.pcl format version 214
INFO in databaseObj.loadBinaryFil


  ATLAS-SUSY-2018-06-eff

  ATLAS-SUSY-2018-32-eff

 weight = None


INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2019-02-eff/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2019-02-eff/db31.pcl in 0.0 secs.
INFO in model.updateParticles() in 393: Loaded 62 BSM particles
INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2019-08-eff/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2019-08-eff/db31.pcl in 0.0 secs.
INFO in model.updateParticles() in 393: Loaded 62 BSM particles



  ATLAS-SUSY-2018-41-eff

 weight = None

  ATLAS-SUSY-2019-02-eff

 weight = None

  ATLAS-SUSY-2019-08-eff


INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2019-09-eff/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /Users/sahananarasimha/smodels-database/13TeV/ATLAS/ATLAS-SUSY-2019-09-eff/db31.pcl in 0.0 secs.
INFO in model.updateParticles() in 393: Loaded 62 BSM particles



  ATLAS-SUSY-2019-09-eff


In [144]:
print(np.array(ana_weights))
print(comb_matrix)

[0.00059875 0.00022952 0.10893801 0.00260334]
[[False  True  True  True]
 [ True False  True False]
 [ True  True False  True]
 [ True False  True False]]


In [145]:
pf = PathFinder(np.array(~comb_matrix, dtype=int), weights=np.array(ana_weights), ignore_subset=True)
print(pf.find_path(top=6))

{0: {'path': [0, 2, 3], 'weight': 0.11214009746215936}, 1: {'path': [2, 3], 'weight': 0.11154134570426773}, 2: {'path': [0, 1, 2], 'weight': 0.10976628453054806}, 3: {'path': [0, 2], 'weight': 0.10953676221863873}, 4: {'path': [1, 2], 'weight': 0.10916753277265642}, 5: {'path': [2], 'weight': 0.1089380104607471}}


### Though the paper mentions that the weights are log(L_bsm/L_sm), the weights do not get registered if they are negative i..e if L_bsm < L_sm. Hence, I have taken weights to be nll ratio. Should clarify it with Jamie